In [1]:
from PIL import Image
import numpy as np

In [2]:
img = Image.open('picasso1.png')

In [3]:
idata = np.asarray(img)
idata_ = np.mean(idata, axis=2) # simple arithmetic mean, as done by the paper
idata = np.uint8(idata_)
img2 = Image.fromarray(idata)

In [4]:
ROWMAX, COLMAX = img2.size
ind_row = np.arange(0, ROWMAX, 1)
ind_col = np.arange(0, COLMAX, 1)

In [5]:
for i in ind_row[21:25]:
    for j in ind_col[45:47]:
        matrix = idata[i:i+2, j:j+2]  # plus 2 makes it actually 2x2 otherwise it's 1x1
        print (matrix)
        #print (matrix.shape)
    print ("====")

[[214 213]
 [216 217]]
[[213 211]
 [217 215]]
====
[[216 217]
 [194 203]]
[[217 215]
 [203 211]]
====
[[194 203]
 [167 179]]
[[203 211]
 [179 194]]
====
[[167 179]
 [155 165]]
[[179 194]
 [165 180]]
====
